In [1]:
import numpy as np
import pandas as pd 

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# AREA

In [3]:
#ben_cost=pd.read_csv('../data/Benefits_Cost_Sharing_PUF.csv',low_memory=False, encoding ='latin1')
#rules=pd.read_csv('../data/Business_Rules_PUF.csv',low_memory=False, encoding ='latin1')
#network=pd.read_csv('../data/Network_PUF.csv',low_memory=False, encoding ='latin1')
#attr=pd.read_csv('../data/Plan_Attributes_PUF.csv',low_memory=False, encoding ='latin1')
#rate=pd.read_csv('../data/Rate_PUF.csv',low_memory=False, encoding ='latin1')
area=pd.read_csv('../data/raw_files/Service_Area_PUF.csv',low_memory=False, encoding ='latin1')

In [4]:
#this gets rid of the Dental Only Plans leaving medical insurance plans then drops the column as superfilous
area=area[area.DentalOnlyPlan != 'Yes']
area.drop('DentalOnlyPlan', axis=1, inplace=True)

area=area.drop(['BusinessYear','SourceName', 'ImportDate'], axis = 1)
#area.info()

In [5]:
# grab states w/ zipcode level plans

zipcodes=area[area.ZipCodes.isnull() == False]

In [6]:
area_FL=area[area.StateCode == 'FL'].copy()
'''
EDA shows FL does not have  plans based on 'PartialCounty', ''ZipCodes', or
PartialCountyJustification' so they are being dropped
'''

area_FL=area_FL.drop(['PartialCounty', 'ZipCodes', 'PartialCountyJustification'], axis = 1)

''' only offer Individual plans so that column is being dropped'''
area_FL=area_FL.drop('MarketCoverage', axis = 1)


In [7]:
in_state= area_FL[area_FL.CoverEntireState== 'Yes']
uniques={} 
for col in area_FL.columns: 
    uniques[col]=area_FL[col].unique()
uniques
county_list=uniques['County']

for county in county_list:
    list1=['FL','16842','FLS001','BlueOptions','Yes']
    list1.append(county)
    area_FL = area_FL.append(pd.Series(list1, index=area_FL.columns ), ignore_index=True)

In [8]:
#dropna has to be done post split
area_FL=area_FL.dropna()
area_FL['IssuerId']=area_FL['IssuerId'].astype(int)
area_FL['County']=area_FL['County'].astype(int)
area_FL.drop(['StateCode','ServiceAreaName','CoverEntireState'],axis=1,inplace=True)

area_FL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 214
Data columns (total 3 columns):
IssuerId         213 non-null int32
ServiceAreaId    213 non-null object
County           213 non-null int32
dtypes: int32(2), object(1)
memory usage: 5.0+ KB


## FIPS

In [ ]:
#this works

fips= pd.read_csv('../data/zipcodes-county-fips-crosswalk/ZIP-COUNTY-FIPS_2017-06.csv',  dtype='str')
fips['STCOUNTYFP']=fips['STCOUNTYFP'].astype(int)
fips['COUNTYNAME'] = fips['COUNTYNAME'].str.replace(r' County', '')
#fips[['County','County_Name']]=fips[['STCOUNTYFP','COUNTYNAME']]
fips = fips[fips.STATE== 'FL']

#fips.reset_index(inplace=True)
'''
FL_zips =pd.DataFrame()
FL_zips[['zipcode','County','County_Name']] = fips[['ZIP','STCOUNTYFP','COUNTYNAME']]

FL_zips.to_csv('../data/clean_files/zipcodes.csv')
'''
FL_county =pd.DataFrame()
FL_county[['County','County_Name']] = fips[['STCOUNTYFP','COUNTYNAME']]


FL_county.sort_values(by=['County'])
FL_county=FL_county.drop_duplicates(keep='first')
result = pd.merge(area_FL,FL_county, how='left', on=['County', 'County'])
#pd.merge(area_FL, FL_county, how='left', left='County',right='County')

result.to_csv('../data/clean_files/area_FL.csv')

result

## combine the two


In [10]:
pd.merge(area_FL, FL_county, how='left', on='County')
area_FL.to_csv('../data/clean_files/area_FL.csv')